## <mark> saving and getting from S3 bucket

In [ ]:
import boto3

from langchain.document_loaders import S3DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

s3 = boto3.client('s3')

# Specify the S3 bucket and directory path
bucket_name = 'bucket_name'
directory_key = 's3_path'

# List objects with a delimiter to get only common prefixes (directories)
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=directory_path, Delimiter='/')

# Extract the common prefixes (directories) from the response
common_prefixes = response.get('CommonPrefixes', [])
    
# Print the directory names
for prefix in common_prefixes:
    print(prefix['Prefix'])

In [ ]:
def create_chromadb_from_s3_docs():
    # Load data from s3
    docs = []
    for key in s3.list_objects_v2(Bucket=bucket_name, Prefix=directory_path, Delimiter='/').get('CommonPrefixes', []):
        loader = S3DirectoryLoader(bucket_name, key['Prefix'])
        docs.extend(loader.load())    

    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings(openai_api_key=open_ai_secret)

    db = Chroma.from_documents(
        texts, embedding=embeddings
    )
    return db

In [ ]:
def save_chromadb_to_s3():
    # Save the vector store data to the S3 bucket
    s3.Object(bucket_name, s3_filename).put(Body=db)